In [1]:
import random
import math

class BGV(object):
    def __init__(self, n, q, B, t):
        self.n = n  # Dimension of the secret vector
        self.q = q  # Modulus
        self.B = B  # Bound on the coefficients of the error vector
        self.t = t  # Scaling factor for ciphertexts
        
        # Generate the public key and the secret key
        self.public_key, self.secret_key = self.keygen()
    
    def keygen(self):
        # Generate a random matrix A and a random vector s
        A = [[random.randint(-self.q//2, self.q//2) for j in range(self.n)] for i in range(self.n)]
        s = [random.randint(-self.q//2, self.q//2) for i in range(self.n)]
        
        # Compute the vector b = As + e, where e is a random error vector
        e = [random.randint(-self.B//2, self.B//2) for i in range(self.n)]
        b = [(sum([A[i][j]*s[j] for j in range(self.n)]) + e[i]) % self.q for i in range(self.n)]
        
        return ((A, b), s)
    
    def encrypt(self, m):
        # Represent the message as a polynomial
        m_poly = [m % self.q]
        
        # Choose a random polynomial r
        r_poly = [random.randint(0, self.q-1) for i in range(self.n)]
        
        # Compute the ciphertext
        c_poly = [(r_poly[i]*self.public_key[0][i][j] + self.public_key[1][i] + m_poly[0]*self.t) % self.q for i in range(self.n) for j in range(self.n)]
        c_matrix = [[c_poly[i*self.n+j] for j in range(self.n)] for i in range(self.n)]
        
        return (c_matrix, c_poly)
    
    def decrypt(self, c):
        # Decode the ciphertext
        c_poly = [c[1][i*self.n+j] for i in range(self.n) for j in range(self.n)]
        
        # Compute c' = c*s/q
        c_prime = [(c_poly[i]*self.secret_key[j]//self.q) % self.q for i in range(self.n**2) for j in range(self.n)]
        
        # Round each entry of c' to the nearest integer
        c_rounded = [round(c_prime[i]) for i in range(self.n**2)]
        
        # Recover the polynomial m(x) by subtracting A*r from c'
        m_poly = [(c_rounded[i] - sum([self.public_key[0][i//self.n][j]*c_rounded[self.n*j+i%self.n] for j in range(self.n)]) % self.q) % self.q for i in range(self.n)]
        
        # Convert the polynomial to a message
        m = sum([m_poly[i]*(self.q**(self.n-i-1)) for i in range(self.n)])
        
        return m

In [2]:
##. n, q, B, t
bgv = BGV(10,23,15,19)

bgv.keygen()

en = bgv.encrypt(20)

In [3]:
en

([[9, 21, 5, 1, 12, 3, 3, 22, 17, 10],
  [19, 20, 17, 16, 2, 13, 8, 7, 2, 14],
  [3, 10, 1, 20, 3, 13, 19, 0, 14, 9],
  [16, 21, 21, 16, 3, 17, 8, 2, 8, 10],
  [6, 18, 6, 14, 1, 15, 15, 17, 8, 21],
  [5, 18, 18, 15, 3, 5, 2, 1, 0, 7],
  [11, 17, 1, 7, 7, 7, 18, 16, 19, 5],
  [1, 6, 5, 18, 4, 7, 7, 20, 20, 16],
  [4, 18, 5, 21, 15, 1, 11, 21, 6, 1],
  [18, 21, 6, 2, 1, 2, 20, 18, 7, 3]],
 [9,
  21,
  5,
  1,
  12,
  3,
  3,
  22,
  17,
  10,
  19,
  20,
  17,
  16,
  2,
  13,
  8,
  7,
  2,
  14,
  3,
  10,
  1,
  20,
  3,
  13,
  19,
  0,
  14,
  9,
  16,
  21,
  21,
  16,
  3,
  17,
  8,
  2,
  8,
  10,
  6,
  18,
  6,
  14,
  1,
  15,
  15,
  17,
  8,
  21,
  5,
  18,
  18,
  15,
  3,
  5,
  2,
  1,
  0,
  7,
  11,
  17,
  1,
  7,
  7,
  7,
  18,
  16,
  19,
  5,
  1,
  6,
  5,
  18,
  4,
  7,
  7,
  20,
  20,
  16,
  4,
  18,
  5,
  21,
  15,
  1,
  11,
  21,
  6,
  1,
  18,
  21,
  6,
  2,
  1,
  2,
  20,
  18,
  7,
  3])

In [4]:
org = bgv.decrypt(en)

In [5]:
org

33572798179387

In [24]:
import random
from math import log2, ceil

# Generate prime number
def generate_prime_number(num_bits):
    while True:
        p = random.getrandbits(num_bits)
        if is_prime(p):
            return p

# Check if number is prime
def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(num ** 0.5) + 1):
        if num % i == 0:
            return False
    return True

# Extended Euclidean algorithm
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

# Modular inverse
def mod_inv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('No modular inverse exists')
    else:
        return x % m

# Encrypt message
def encrypt(msg, p, q, N, B, a):
    # Pad the message with zeros
    msg = [ord(c) for c in msg]
    num_zeros = (N.bit_length() // 8) - len(msg)
    msg.extend([0] * num_zeros)
    
    # Convert the message to a polynomial
    m = sum([msg[i] * B ** i for i in range(len(msg))])
    
    # Encrypt the polynomial
    e1 = random.randint(0, N - 1)
    e2 = random.randint(0, N - 1)
    c1 = pow(a, e1, N) * pow(q, e2, N) % N
    c2 = (m * pow(B, e1, N) + e2) % N
    return (c1, c2)

# Decrypt message
def decrypt(c, p, q, N, B):
    c1, c2 = c
    a_inv = mod_inv(pow(p, ceil(log2(B)), N), N)
    m = ((c2 % N) * mod_inv(pow(c1, ceil(log2(B)) * p - 1, N), N)) % N
    m = [m % B ** i // B ** (i - 1) for i in range(ceil(log2(B)), len(bin(m)) - 1, ceil(log2(B)) * -1)][::-1]
    return ''.join([chr(c) for c in m]).strip('\x00')

# Example usage
if __name__ == '__main__':
    # Generate primes
    p = generate_prime_number(32)
    print(p)
    q = generate_prime_number(32)
    
    # Compute public key parameters
    N = p * q
    B = 2 ** 32
    a = random.randint(0, N - 1)
    
    # Encrypt message
    msg = 'Hello, world!'
    ciphertext = encrypt(msg, p, q, N, B, a)
    
    # Decrypt message
    plaintext = decrypt(ciphertext, p, q, N, B)
    
    # Print results
    print('Message:', msg)
    print('Ciphertext:', ciphertext)
    print('Plaintext:', plaintext)

2147651269


Exception: No modular inverse exists

In [25]:
import random
from math import ceil, log2

# Function to generate a random vector with integer entries in the range [-q/2, q/2]
def random_vector(q, n):
    return [random.randint(-q//2, q//2) for i in range(n)]

# Function to generate a random matrix with integer entries in the range [-q/2, q/2]
def random_matrix(q, m, n):
    return [[random.randint(-q//2, q//2) for j in range(n)] for i in range(m)]

# Function to compute the dot product of two vectors
def dot_product(v1, v2):
    return sum(v1[i]*v2[i] for i in range(len(v1)))

# Function to add two vectors
def add_vectors(v1, v2):
    return [v1[i] + v2[i] for i in range(len(v1))]

# Function to multiply a vector by a scalar
def scalar_multiply(v, s):
    return [s*v[i] for i in range(len(v))]

# Function to compute the modulus of a vector
def mod_vector(v, q):
    return [v[i] % q for i in range(len(v))]

# Function to compute the modulus of a matrix
def mod_matrix(A, q):
    return [[A[i][j] % q for j in range(len(A[0]))] for i in range(len(A))]

# Function to generate public and secret keys for the BGV scheme
def generate_keys(n, q, B):
    # Generate a random matrix A and vector s
    A = random_matrix(q, n, n)
    s = random_vector(q, n)

    # Compute b = As + e, where e is a random vector with entries in the range [-B/2, B/2]
    e = random_vector(B, n)
    b = add_vectors(scalar_multiply(A, s), e)

    # Return public key (A, b) and secret key s
    return (mod_matrix(A, q), mod_vector(b, q)), mod_vector(s, q)

# Generate keys for n=2, q=13, B=5
public_key, secret_key = generate_keys(2, 13, 5)

# Print the keys
print("Public key:")
print(public_key)
print("Secret key:")
print(secret_key)

TypeError: can't multiply sequence by non-int of type 'list'